<a href="https://colab.research.google.com/github/anik314159/RVIL-CDIP/blob/main/codeforsubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy 
import nltk
import re
import cv2
import pathlib
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:

pred_labels = []

#populate this

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_data_root = pathlib.Path('/content/drive/MyDrive/IndoML_Datathon_2022/validation/validation')
text_data_root = pathlib.Path('/content/drive/MyDrive/IndoML_Datathon_2022/validation/text_data')

In [ ]:
def get_text_from_path(path):
    with open(path) as f:
        lines = f.readlines()
        lines  = ' '.join(lines)
        f.close()
    return lines

In [ ]:
def get_file_paths_and_labels(img_data_root, text_data_root):
    img_paths = [str(path) for path in img_data_root.glob('**/*.tif')]
    text_paths = [str(path) for path in text_data_root.glob('**/*.txt')]
    
    return img_paths, text_paths



In [ ]:
text_data = []
img_paths , text_paths =  get_file_paths_and_labels(img_data_root, text_data_root);
for path in text_paths:
  text_data.append(get_text_from_path(path))


In [ ]:
df = pd.DataFrame(list(zip(text_paths, text_data, img_paths)),
               columns =['text_paths','texts', 'img_paths'])

# Merged Dataframe
df.head()

,text_paths,texts,img_paths
0,/content/drive/MyDrive/IndoML_Datathon_2022/va...,"\n \n M@§ Tobacco\n \n industry _,\n firing ...",/content/drive/MyDrive/IndoML_Datathon_2022/va...
1,/content/drive/MyDrive/IndoML_Datathon_2022/va...,1820\n \n Inhalation of Sidestream Cigarette\n...,/content/drive/MyDrive/IndoML_Datathon_2022/va...
2,/content/drive/MyDrive/IndoML_Datathon_2022/va...,cAI\n \n IESE\n \n 4KWUZ82105\n \n LABORATORIE...,/content/drive/MyDrive/IndoML_Datathon_2022/va...
3,/content/drive/MyDrive/IndoML_Datathon_2022/va...,SPECIFICATION CHANGE PROPOSAL NO._89-395 _\n \...,/content/drive/MyDrive/IndoML_Datathon_2022/va...
4,/content/drive/MyDrive/IndoML_Datathon_2022/va...,2083779468\n \n \n \n i\n um Apuy puz\n \n On...,/content/drive/MyDrive/IndoML_Datathon_2022/va...


In [ ]:

def preprocess(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = preprocessed_string.replace('_',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    preprocessed_string =  " ".join(preprocessed_string.split())
    return preprocessed_string

In [ ]:


nlp = spacy.load("en_core_web_sm")

from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words("english")

def get_stopwords(file_path):
    with open(file_path, "r") as fp:
        content = fp.read()
        stops = content.split("\n")
        stops = stops[:-1]
        fp.close()
        return stops

from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words("english")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'count play home'

In [ ]:
df['texts'] = [preprocess(str(this_text)) for this_text in df['texts']]
df['texts'] = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in df['texts']]
df.head()

,text_paths,texts,img_paths
0,/content/drive/MyDrive/IndoML_Datathon_2022/va...,tobacco industry fire back synar say I keith w...,/content/drive/MyDrive/IndoML_Datathon_2022/va...
1,/content/drive/MyDrive/IndoML_Datathon_2022/va...,1820 inhalation sidestream cigarette smoke acc...,/content/drive/MyDrive/IndoML_Datathon_2022/va...
2,/content/drive/MyDrive/IndoML_Datathon_2022/va...,cai iese 4kwuz82105 laboratory corporation,/content/drive/MyDrive/IndoML_Datathon_2022/va...
3,/content/drive/MyDrive/IndoML_Datathon_2022/va...,specification change proposal 89395 brand kent...,/content/drive/MyDrive/IndoML_Datathon_2022/va...
4,/content/drive/MyDrive/IndoML_Datathon_2022/va...,2083779468 I um apuy puz sdjey deysiy ul sayse...,/content/drive/MyDrive/IndoML_Datathon_2022/va...


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len = 500 
# Fit the tokenizer
tokenizer = Tokenizer(num_words = 65000)#hp num_words --> no.of unique words
tokenizer.fit_on_texts(df['texts']) # --> check for np.unique on all documents and sum

# sequence the input corpus and add zero padding upto 500 word


test_sequence = tokenizer.texts_to_sequences(df['texts'])
test_padded = pad_sequences(test_sequence, maxlen = max_len, truncating = "post", padding = "post" )


test_tensor = [tf.convert_to_tensor(test_padded[i]) for i in range(test_padded.shape[0])]
df['texts_embedding'] = test_tensor   


In [ ]:
def dataframe_to_dataset(dataframe):
    d = {}
    embed_tensor = []
    for i in dataframe['texts_embedding']:
        embed_tensor.append(tf.convert_to_tensor(i))

    img_data_array = []
    for this_path in dataframe['img_paths']:
        image= cv2.imread(this_path, cv2.COLOR_BGR2RGB)
        image=cv2.resize(image, (224, 224))
        image=np.array(image)
        #image = image.astype('float32')
        #image /= 255 
        image = np.stack((image,)*3, axis=-1)
        img_data_array.append(tf.convert_to_tensor(image))

    d['texts_embedding'] = embed_tensor
    d['img_paths'] = img_data_array

    ds = tf.data.Dataset.from_tensor_slices(d)
    return ds

In [ ]:
@tf.function
def preprocess_image(image):
    image = tf.image.per_image_standardization(image)
    return image

@tf.function
def preprocess_text(sample):
    pass

@tf.function
def preprocess_text_and_image(sample):
    image = sample["img_paths"]
    text = sample['texts_embedding']
    return {"image_inputs": image,  "text_inputs": text}
   

In [ ]:
batch_size = 128
auto = tf.data.experimental.AUTOTUNE

def prepare_dataset(dataframe, training = True):
    ds = dataframe_to_dataset(dataframe)
    if training:
        ds = ds.shuffle(len(df))
    ds = ds.map(lambda x: (preprocess_text_and_image(x))).cache()
   # ds = ds.map(lambda x,y: (preprocess_text_and_image(x), y)).cache()
    ds = ds.batch(batch_size).prefetch(auto)
    return ds

In [ ]:
test_ds = prepare_dataset(df, False)

In [ ]:
pred_labels = model.predict(test_ds)

In [ ]:


data_root = pathlib.Path('/content/drive/MyDrive/IndoML_Datathon_2022/validation/validation')
image_paths = []
label = []
for item in data_root.iterdir():
  label = int(item.split('/')[-1].split('.')[0])
  label_col.append(label)



In [ ]:
import pandas as pd

df = pd.DataFrame(list(zip(label_col,pred_labels)),colums = ['feature','labels'])


In [ ]:
df.to_csv('/content/drive/MyDrive/IndoML_Datathon_2022/validation')